In [9]:
# Cài đặt thư viện
!pip install fastapi uvicorn nest-asyncio pyngrok kagglehub streamlit requests > /dev/null

import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import joblib
import kagglehub
from fastapi import FastAPI
from pydantic import BaseModel, confloat
import threading
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import time
import os
os.system("killall ngrok || true")

# ------------------------------------------
# Huấn luyện và lưu mô hình
path = kagglehub.dataset_download("uciml/iris")
csv_path = os.path.join(path, "Iris.csv")
df = pd.read_csv(csv_path)

X = df.drop(columns=["Id", "Species"])
y = df["Species"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
joblib.dump(knn, "knn_model.pkl")

model = joblib.load("knn_model.pkl")
classes = model.classes_

# ------------------------------------------
# FASTAPI
app = FastAPI()

class IrisInput(BaseModel):
    SepalLengthCm: confloat(ge=4.3, le=7.9)
    SepalWidthCm: confloat(ge=2.0, le=4.4)
    PetalLengthCm: confloat(ge=1.0, le=6.9)
    PetalWidthCm: confloat(ge=0.1, le=2.5)

@app.post("/predict")
def predict(data: IrisInput):
    df = pd.DataFrame([data.dict()])
    pred = model.predict(df)[0]
    probas = model.predict_proba(df)[0]
    proba_dict = dict(zip(classes, [round(float(p), 3) for p in probas]))
    return {"prediction": pred, "probabilities": proba_dict}

# ------------------------------------------
# Tạo file Streamlit
streamlit_code = '''
import streamlit as st
import requests

with open("api_url.txt") as f:
    API_URL = f.read().strip() + "/predict"

st.title("Dự đoán Hoa Iris")
st.write("Nhập các thông số rồi bấm **Dự đoán**")

sl = st.number_input("Sepal Length (cm) (4.3–7.9)", min_value=4.3, max_value=7.9, value=5.1)
sw = st.number_input("Sepal Width (cm) (2.0–4.4)", min_value=2.0, max_value=4.4, value=3.5)
pl = st.number_input("Petal Length (cm) (1.0–6.9)", min_value=1.0, max_value=6.9, value=1.4)
pw = st.number_input("Petal Width (cm) (0.1–2.5)", min_value=0.1, max_value=2.5, value=0.2)

if st.button("Dự đoán"):
    data = {
        "SepalLengthCm": sl,
        "SepalWidthCm": sw,
        "PetalLengthCm": pl,
        "PetalWidthCm": pw
    }
    st.write(f"📡 Gửi đến API: `{API_URL}` …")
    res = requests.post(API_URL, json=data)
    if res.status_code == 200:
        r = res.json()
        st.success(f"Loài dự đoán: **{r['prediction']}**")
        st.write("Mức độ tự tin của mô hình:")
        probas = r["probabilities"]
        # Hiển thị từng dòng
        for cls, prob in sorted(probas.items(), key=lambda x: -x[1]):
            pct = int(round(prob * 100))
            st.write(f"Mô hình tự tin **{pct}%** là loài **{cls}**")
    else:
        st.error(f"Lỗi: {res.status_code}")
'''

with open("iris_app.py", "w") as f:
    f.write(streamlit_code)

# ------------------------------------------
# Chạy song song
def run_api():
    nest_asyncio.apply()
    uvicorn.run(app, host="0.0.0.0", port=8000)

def run_streamlit():
    os.system("streamlit run iris_app.py &")

# Mở ngrok cho FastAPI
ngrok.set_auth_token("2zS1ix1LExFP3PbwbOb4zUtovvE_7TmYVGJoGxEp5xCUsD9rA")
api_tunnel = ngrok.connect(8000)
api_url = api_tunnel.public_url
with open("api_url.txt", "w") as f:
    f.write(api_url)

# Mở ngrok cho Streamlit
streamlit_tunnel = ngrok.connect(8501)
streamlit_url = streamlit_tunnel.public_url

# Chạy API + Streamlit
threading.Thread(target=run_api).start()
time.sleep(5)  # đợi API lên
threading.Thread(target=run_streamlit).start()
time.sleep(10)  # đợi Streamlit lên

print("API URL: ", api_url + "/docs")
print("Giao diện Web: ", streamlit_url)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-16' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1536, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-8-2987315909.py", line 101, in

API URL:  https://653e-35-223-107-159.ngrok-free.app/docs
Giao diện Web:  https://b5f7-35-223-107-159.ngrok-free.app
